This is the analysis for the ConvX (a Udacity Capstone Project). Before starting, be sure to follow the instructions in the README for the project on github (https://github.com/justiniann/ConvX).

This code was originally run on a personal desktop computer. The specs were...

CPU: Intel i5 6600K, 
GPU: Nvidia 1060 GTX, 
RAM: 16GB

I highly recommend that anyone attempting to run this code on the full dataset use hardware that is comparable or better.

First, we need to load our base model. We will also define a few variables that we will need later.

In [ ]:
import os
import random
from keras.applications import ResNet50
from keras.layers import Dropout, GlobalAveragePooling2D, Dense, Flatten
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from keras.models import model_from_json
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score, fbeta_score, confusion_matrix
import matplotlib.pyplot as plt
from convx_utils import *

base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(512, 512, 3))
target_image_size = (512, 512)
batch_size = 32
transfer_learning_epochs = 500
fine_tuning_epochs = 5
fine_tuning_layers_to_train = 10

We'll also define some paths that we will need later for saving various results as we run through our analysis

In [ ]:
# The following are directories used for reading/saving data
RES_PATH = "..{0}..{0}resources{0}".format(os.path.sep)
IMG_PATH = "..{0}..{0}images{0}".format(os.path.sep)
BOTTLENECK_PATH = "..{0}..{0}bottleneck{0}".format(os.path.sep)
SAVE_PATH = "..{0}..{0}saved_models{0}".format(os.path.sep)
TRAIN_PATH = os.path.join(IMG_PATH, "train")
VAL_PATH = os.path.join(IMG_PATH, "validation")
TEST_PATH = os.path.join(IMG_PATH, "test")

model_name = "convx_model"  # The directory all data will be saved in will be named whatever this value is.
models_save_directory = os.path.join(SAVE_PATH, model_name)
build_dir_path(models_save_directory)  # build the directory structure we need for saving results

We haven't included the top layer because we are going to build and train the top layer ourselves. This is known as transfer learning, and it is the first major step in training our model.

Before starting that, however, we need to get a few variables we are going to need durring processing

In [ ]:
def count_files(root_dir):
    return sum([len(files) for r, d, files in os.walk(root_dir)])

def get_iterations_per_epoch(total_images, batch_size):
    return np.ceil(total_images / batch_size)

healthy_train_images = count_files(os.path.join(TRAIN_PATH, "healthy"))
unhealthy_train_images = count_files(os.path.join(TRAIN_PATH, "unhealthy"))
healthy_validation_images = count_files(os.path.join(VAL_PATH, "healthy"))
unhealthy_validation_images = count_files(os.path.join(VAL_PATH, "unhealthy"))

num_training_steps = get_iterations_per_epoch((healthy_train_images + unhealthy_train_images), batch_size)
num_validation_steps = get_iterations_per_epoch((healthy_validation_images + unhealthy_validation_images), batch_size)

data_generator = ImageDataGenerator(
    rescale=1. / 255
)

For efficiency, we are going to get and save the bottleneck features for this model before we start with the transfer learning. By obtaining and saving these once, we can avoid having to run every image through the entire network durring every epoch. 

In [ ]:
bottleneck_file_path = os.path.join(BOTTLENECK_PATH, model_name)
train_bottleneck_file = os.path.join(bottleneck_file_path, "train.npy")
validation_bottleneck_file = os.path.join(bottleneck_file_path, "validation.npy")
    
# Extract bottleneck features if they have not been already
if not os.path.exists(bottleneck_file_path):
    train_generator = data_generator.flow_from_directory(
        TRAIN_PATH,
        target_size=target_image_size,
        batch_size=batch_size,
        class_mode=None,
        shuffle=False
    )
    
    bottleneck_features_train = base_model.predict_generator(train_generator, num_training_steps)
    build_dir_path(bottleneck_file_path)
    np.save(open(train_bottleneck_file, 'wb'), bottleneck_features_train)
    
    validation_path_generator = data_generator.flow_from_directory(
        VAL_PATH,
        target_size=target_image_size,
        batch_size=batch_size,
        class_mode=None,
        shuffle=False
    )
    
    bottleneck_features_validation = base_model.predict_generator(validation_path_generator, num_validation_steps)
    np.save(open(validation_bottleneck_file, 'wb'), bottleneck_features_validation)

With the bottleneck features established, we can start transfer learning.

In [ ]:
def build_fully_connected_top_layer(connecting_shape):
    top_layers = Sequential()
    top_layers.add(GlobalAveragePooling2D(input_shape=connecting_shape))
    top_layers.add(Dense(512, activation='tanh'))
    top_layers.add(Dropout(0.4))
    top_layers.add(Dense(2, activation='softmax'))
    return top_layers

def compile_model(model):
    model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

# load training data
train_data = np.load(open(train_bottleneck_file, 'rb'))
train_labels = to_categorical(np.array(([0] * healthy_train_images) + ([1] * unhealthy_train_images)),
                              num_classes=2)
# load validation data
validation_data = np.load(open(validation_bottleneck_file, 'rb'))
validation_labels = to_categorical(np.array([0] * healthy_validation_images + [1] * unhealthy_validation_images),
                                   num_classes=2)

top_layer = build_fully_connected_top_layer(train_data.shape[1:])

compile_model(top_layer)

transfer_history = top_layer.fit(train_data, train_labels,
                                  epochs=transfer_learning_epochs,
                                  batch_size=batch_size,
                                  validation_data=(validation_data, validation_labels),
                                  shuffle=True,
                                  verbose=1)

top_layers_weights_path = os.path.join(models_save_directory, "transfer_learning_weights_v2.h5")
top_layer.save_weights(top_layers_weights_path)

Transfer learning has been completed and the results have been saved! We can now combine the base model with our newly trained top layer and analyze the results.

In [ ]:
def display_history(history):
    # summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('Model Binary Accuracy')
    plt.ylabel('Accuracy')
    plt.ylim(0.4,0.8)
    plt.xlabel('Epoch')
    plt.xlim(0,500)
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Binary Cross-Entropy Loss')
    plt.ylabel('Loss')
    plt.ylim(0.6,0.8)
    plt.xlabel('Epoch')
    plt.xlim(0,500)
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    
display_history(transfer_history)

We can further improve our results by finetuning the model. Using the transfer learning model that we have already trained, we can 'unfreeze' a few of the layers from the base model. This will allow them to be trained, giving us an even better fit on the data. 

In [ ]:
top_layer = build_fully_connected_top_layer(base_model.output_shape[1:])
top_layer.load_weights(top_layers_weights_path)
convx_model = Model(inputs=base_model.input, outputs=top_layer(base_model.output))

for layer in convx_model.layers[:len(convx_model.layers) - fine_tuning_layers_to_train]:
    layer.trainable = False

compile_model(convx_model)

train_generator = data_generator.flow_from_directory(
    TRAIN_PATH,
    target_size=target_image_size,
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = data_generator.flow_from_directory(
    VAL_PATH,
    target_size=target_image_size,
    batch_size=batch_size,
    class_mode='categorical')

fine_tune_history = convx_model.fit_generator(
    train_generator,
    steps_per_epoch=num_training_steps,
    epochs=fine_tuning_epochs,
    validation_data=validation_generator,
    validation_steps=num_validation_steps,
    verbose=1
)

convx_model.save_weights(os.path.join(models_save_directory, "best_model_weights_v2.h5"))

with open(os.path.join(models_save_directory, "best_model_v2.json".format(model_name)), "w") as json_file:
    json_file.write(convx_model.to_json())

Once again, we will plot the history of our training.

In [ ]:
display_history(fine_tune_history)

Our model has now been fine tuned and the training process is complete! Lets evaluate the results. While we are also going to look at accuracy, our primary metric of evaluation is going to be f-beta, with a beta score of three. With this, we will get to see which models fit the data well while giving more weight to good recall. 

And now, the evaluation.

In [ ]:
convx_model = None
with open(os.path.join(models_save_directory, "best_model.json"), 'r') as model_file:
    convx_model = model_from_json(model_file.read())
    
convx_model.load_weights(os.path.join(models_save_directory, "best_model_weights.h5"))
compile_model(convx_model)

healthy_test_images = count_files(os.path.join(TEST_PATH, "healthy"))
unhealthy_test_images = count_files(os.path.join(TEST_PATH, "unhealthy"))
test_iteration_count = get_iterations_per_epoch((healthy_test_images + unhealthy_test_images), batch_size)

# load test data
test_labels = np.array(([0] * healthy_test_images) + ([1] * unhealthy_test_images))
formatted_test_labels = to_categorical(test_labels, num_classes=2)

test_generator = data_generator.flow_from_directory(
    TEST_PATH,
    target_size=target_image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
)

raw_predictions = convx_model.predict_generator(test_generator, test_iteration_count, verbose=1)
predicted_labels = np.argmax(raw_predictions, axis=1)

accuracy = accuracy_score(test_labels, predicted_labels)
print("Accuracy: {}".format(accuracy))

f3_score = fbeta_score(test_labels, predicted_labels, 3)
print("F3 Score: {}".format(f3_score))

conf_matrix = confusion_matrix(test_labels, predicted_labels)
print("Confusion Matrix: \n{}".format(conf_matrix))

Now, if we give more weight to good recall over good precision, the numbers are much different.

In [ ]:
def predict_with_weight(raw_predictions, p):
    res = []
    for pred in raw_predictions:
        res.append(0 if pred[0] > p else 1)
    return res

x = []
true_negatives = []
false_negatives = []
for i in range(50, 100):
    weight = i/100
    x.append(weight)
    weighted_predictions = predict_with_weight(raw_predictions, weight)
    conf_matrix = confusion_matrix(test_labels, weighted_predictions)
    true_negatives.append(conf_matrix[0][0])
    false_negatives.append(conf_matrix[1][0])
    
plt.plot(x, true_negatives, 'b')
plt.plot(x, false_negatives, 'r')
plt.title('True Negatives vs False Negatives')
plt.ylabel('Count')
plt.xlabel('Unhealthy Class Weight')
plt.legend(['True Negatives', 'False Negatives'], loc='upper right')
plt.show()

